# Import libraries

In [13]:
import pandas as pd
import numpy as np
import os

# Load data

In [14]:
# define the path and assign dataframes to the "df" variables

path = r'C:\Users\Jakov\2023 Instacart Basket Analysis'

df_ords_prods = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_with_derived_columns.pkl'))

# 02.

In [16]:
# create the new column "max_order" and fill it with max('order_number') for each user

df_ords_prods['max_order'] = df_ords_prods.groupby(['user_id'])['order_number'].transform(np.max)

In [17]:
df_ords_prods.head(10)

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,orders_day_of_a_week,order_hour_of_day,days_since_prior_order,first_order_ever,product_name,aisle_id,department_id,prices,_merge,price_range,busiest_day,busiest_days,busiest_period_of_day,max_order
0,2,33120,1,1,202279,3,5,9,8.0,False,Organic Egg Whites,86,16,11.3,both,Mid-range product,Regularly busy,Regular days,Most orders,8
1,26,33120,5,0,153404,2,0,16,7.0,False,Organic Egg Whites,86,16,11.3,both,Mid-range product,Busiest day,Busiest days,Most orders,32
2,120,33120,13,0,23750,11,6,8,10.0,False,Organic Egg Whites,86,16,11.3,both,Mid-range product,Regularly busy,Regular days,Average orders,13
3,327,33120,5,1,58707,21,6,9,8.0,False,Organic Egg Whites,86,16,11.3,both,Mid-range product,Regularly busy,Regular days,Most orders,30
4,390,33120,28,1,166654,48,0,12,9.0,False,Organic Egg Whites,86,16,11.3,both,Mid-range product,Busiest day,Busiest days,Most orders,62
5,537,33120,2,1,180135,15,2,8,3.0,False,Organic Egg Whites,86,16,11.3,both,Mid-range product,Regularly busy,Regular days,Average orders,99
6,582,33120,7,1,193223,6,2,19,10.0,False,Organic Egg Whites,86,16,11.3,both,Mid-range product,Regularly busy,Regular days,Average orders,40
7,608,33120,5,1,91030,11,3,21,12.0,False,Organic Egg Whites,86,16,11.3,both,Mid-range product,Regularly busy,Slowest days,Fewest orders,38
8,623,33120,1,1,37804,63,3,12,3.0,False,Organic Egg Whites,86,16,11.3,both,Mid-range product,Regularly busy,Slowest days,Most orders,63
9,689,33120,4,1,108932,16,1,13,3.0,False,Organic Egg Whites,86,16,11.3,both,Mid-range product,Regularly busy,Busiest days,Most orders,24


# 03. Since the subset has only first 1000000 rows, it can happen that the same user has lower 'max_order' value in the subset than it has in the complete dataset.

# 04.

In [18]:
# insert a loyalty flag in the "max_order" column 

df_ords_prods.loc[df_ords_prods['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df_ords_prods.loc[(df_ords_prods['max_order'] <= 40) & (df_ords_prods['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df_ords_prods.loc[df_ords_prods['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

# 05.

In [21]:
# Create a view with a data grouped by "loyalty_flag" column and aggregated "prices" variable with count, mean, min and max functions

df_ords_prods.groupby('loyalty_flag').agg({'prices': ['count', 'mean', 'min', 'max']})

prices                         
                     count       mean  min      max
loyalty_flag                                       
Loyal customer    10284093  10.386336  1.0  99999.0
New customer       6243990  13.294670  1.0  99999.0
Regular customer  15876776  12.495717  1.0  99999.0

# Loyal customers tend to purchase products with the lowest price on average, but make the most purchases (per user) among customer groups.

# 06.

In [29]:
# Create a new column "avg_price" filled with average users' spending per item

df_ords_prods['avg_price'] = round(df_ords_prods.groupby(['user_id'])['prices'].transform(np.mean), 2)

In [31]:
# fill the "spending_flag" column with "Low spender" value if average users' spending is lower than 10

df_ords_prods.loc[df_ords_prods['avg_price'] < 10, 'spending_flag'] = 'Low spender'

In [32]:
# fill the "spending_flag" column with "High spender" value if average users' spending is lower than or equal to 10

df_ords_prods.loc[df_ords_prods['avg_price'] >= 10, 'spending_flag'] = 'High spender'

# 07.

In [37]:
# Create a new column "med_freq_order" filled with users' median of "days_since_prior_order" variable

df_ords_prods['med_freq_order'] = df_ords_prods.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [39]:
# fill the "Frequency_flag" column with "Non_frequent customer" value if condition is met

df_ords_prods.loc[df_ords_prods['med_freq_order'] > 20, 'Frequency_flag'] = 'Non-frequent customer'

# fill the "Frequency_flag" column with "Regular customer" value if condition is met

df_ords_prods.loc[(df_ords_prods['med_freq_order'] > 10) & (df_ords_prods['med_freq_order'] <= 20), 'Frequency_flag'] = 'Regular customer'

# fill the "Frequency_flag" column with "Frequent customer" value if condition is met

df_ords_prods.loc[df_ords_prods['med_freq_order'] <= 10, 'Frequency_flag'] = 'Frequent customer'

# 09.

In [41]:
df_ords_prods.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_with_aggregations.pkl'))